<a href="https://colab.research.google.com/github/MarioPrado1148/DSWP_Aluno_Mario/blob/main/AttendeeReport_14092020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import Markdown

In [ ]:
def d_ht(obj, nh=5, nt=5):
      display(obj.head(nh), obj.tail(nt), obj.shape)

In [ ]:
def d_pd(obj, filtros=None, n=1):
    if not filtros:
        display(obj.iloc[list(range(0,n))+(list(range(-n,0)))], obj.shape)
    else:
        for filtro in filtros:
            display(obj.loc[filtro], Markdown('<br/>'))
        display(obj.shape)     

In [ ]:
def exemplo_linha(frame, iloc=0):
    df_info = pd.DataFrame(dict(valor_exemplo=frame.iloc[iloc].copy()))
    df_info['dtype_coluna'] = frame.dtypes.map(lambda x: x.name)
    df_info['classe_valor'] = df_info['valor_exemplo'].map(lambda x: x.__class__.__name__)
    df_info.index.name = 'nome_coluna'
    return df_info

In [ ]:
csv_presenca = "AttendeeReport 14-09-2020.csv"
data_evento = pd.to_datetime('2020-09-14')
hora_inicio = pd.to_timedelta('14:00:00')
hora_termino = pd.to_timedelta('18:00:00')
dct_horarios = {
    'Joined': data_evento + hora_inicio, 
    'Left':   data_evento + hora_termino
}
dct_horarios

{'Joined': Timestamp('2020-09-14 14:00:00'),
 'Left': Timestamp('2020-09-14 18:00:00')}

In [ ]:
df_presenca = pd.read_csv(csv_presenca).apply(lambda sr: sr.str.strip())
d_pd(df_presenca,n=4)

,Session Id,Participant Id,Full Name,UserAgent,UTC Event Timestamp,Action,Role
0,a82ad53c-0130-49a6-abd6-7bf714973e70,leandro.filoni@rfb.gov.br,,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,8/7/2020 6:29:13 PM,Joined,Attendee
1,790ecec1-bdbc-4b8a-87bd-74e868d2c722,Fabio.Castilho@rfb.gov.br,,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,8/12/2020 2:53:42 PM,Joined,Attendee
2,f8074bde-92b5-47f7-a6e9-4acaedb79e05,leandro.filoni@rfb.gov.br,,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,8/17/2020 5:58:08 PM,Joined,Attendee
3,c7398489-de13-470f-8f77-a1f9ecd8d9d9,leandro.filoni@rfb.gov.br,,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,8/18/2020 8:57:39 PM,Joined,Attendee
11633,4a0c390b-2e42-4b61-a6aa-63f674015fef,anderson.martins@rfb.gov.br,,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,9/15/2020 12:02:55 PM,Left,Attendee
11634,954fdb5b-0eec-4977-a0ae-a44a7c605f30,mark.tollemache@rfb.gov.br,,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,9/15/2020 12:10:20 PM,Left,Attendee
11635,28de2425-1ca6-4cc1-8f22-bd4c465310ee,thiago.pizzolato@rfb.gov.br,,SignalR (lang=Java; os=linux; version=2.0),9/15/2020 12:21:56 PM,Left,Attendee
11636,014ba329-e88a-4f66-94fa-e07406ef288b,vicente.battista@rfb.gov.br,,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,9/15/2020 12:34:48 PM,Left,Attendee


(11637, 7)

In [ ]:
exemplo_linha(df_presenca)

,valor_exemplo,dtype_coluna,classe_valor
nome_coluna,,,
Session Id,a82ad53c-0130-49a6-abd6-7bf714973e70,object,str
Participant Id,leandro.filoni@rfb.gov.br,object,str
Full Name,,object,str
UserAgent,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,object,str
UTC Event Timestamp,8/7/2020 6:29:13 PM,object,str
Action,Joined,object,str
Role,Attendee,object,str


In [ ]:
df_presenca.columns = (
    df_presenca.columns
    .str.strip()
    .str.replace('UTC Event ', '')
    .str.replace(' ', '_')
    .str.replace('(?<=[a-z])(?=[A-Z])', '_')
    .str.lower()
)
d_pd(df_presenca)

,session_id,participant_id,full_name,user_agent,timestamp,action,role
0,a82ad53c-0130-49a6-abd6-7bf714973e70,leandro.filoni@rfb.gov.br,,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,8/7/2020 6:29:13 PM,Joined,Attendee
11636,014ba329-e88a-4f66-94fa-e07406ef288b,vicente.battista@rfb.gov.br,,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,9/15/2020 12:34:48 PM,Left,Attendee


(11637, 7)

In [ ]:
df_completo = (
    df_presenca
    .assign(timestamp = pd.to_datetime(df_presenca.timestamp) - pd.to_timedelta('3:00:00'))
    .set_index(['participant_id', 'session_id', 'action'])
    ['timestamp']
    .unstack('action')
    .fillna(dct_horarios)
)
d_pd(df_completo,n=2)

,action,Joined,Left
participant_id,session_id,,
ANA.BEJAR@rfb.gov.br,baf42d52-7835-49f1-a408-87de78ab0664,2020-09-14 14:30:11,2020-09-14 17:53:09
Abrahao.Lima@rfb.gov.br,09bd90ff-1447-4052-b309-025227fe90ca,2020-09-14 15:12:49,2020-09-14 18:13:08
yuri.godoi@rfb.gov.br,4a857e53-8d7c-4e07-b1e7-2b4e3ea85695,2020-09-14 13:57:27,2020-09-14 16:57:06
zilda.h.pires@rfb.gov.br,cb0f5a7a-e58e-4418-8e38-2f336e88f498,2020-09-14 14:00:03,2020-09-14 18:01:21


(5823, 2)

In [ ]:
df_completo.isna().sum()

action
Joined    0
Left      0
dtype: int64

In [ ]:
df_completo.describe()

action,Joined,Left
count,5823,5823
unique,4377,4072
top,2020-09-14 14:01:16,2020-09-14 17:59:10
freq,7,32
first,2020-08-07 15:29:13,2020-08-07 15:36:18
last,2020-09-15 09:10:16,2020-09-15 09:45:13


In [ ]:
sr_inicio = pd.Series(dct_horarios['Joined'], index=df_completo.index)
sr_termino = pd.Series(dct_horarios['Left'], index=df_completo.index)
df_completo['inicio'] = np.minimum(np.maximum(df_completo.Joined, sr_inicio), sr_termino)
df_completo['termino'] = np.maximum(np.minimum(df_completo.Left, sr_termino), sr_inicio)
d_ht(df_completo.sort_values(['Joined','Left']))

action                                                                      Joined  \
participant_id            session_id                                                 
leandro.filoni@rfb.gov.br a82ad53c-0130-49a6-abd6-7bf714973e70 2020-08-07 15:29:13   
Fabio.Castilho@rfb.gov.br 790ecec1-bdbc-4b8a-87bd-74e868d2c722 2020-08-12 11:53:42   
leandro.filoni@rfb.gov.br f8074bde-92b5-47f7-a6e9-4acaedb79e05 2020-08-17 14:58:08   
                          c7398489-de13-470f-8f77-a1f9ecd8d9d9 2020-08-18 17:57:39   
                          8d822132-3478-4bbf-a394-b976881db90a 2020-08-19 17:18:24   

action                                                                        Left  \
participant_id            session_id                                                 
leandro.filoni@rfb.gov.br a82ad53c-0130-49a6-abd6-7bf714973e70 2020-08-07 15:36:18   
Fabio.Castilho@rfb.gov.br 790ecec1-bdbc-4b8a-87bd-74e868d2c722 2020-08-12 11:54:57   
leandro.filoni@rfb.gov.br f8074bde-92b5-47f7-a6e9-4acaedb79e05 2020-08-17 14:58:26   
                          c7398489-de13-470f-8f77-a1f9ecd8d9d9 2020-08-18 17:57:42   
                          8d822132-3478-4bbf-a394-b976881db90a 2020-08-19 17:18:26   

action                                                                      inicio  \
participant_id            session_id                                                 
leandro.filoni@rfb.gov.br a82ad53c-0130-49a6-abd6-7bf714973e70 2020-09-14 14:00:00   
Fabio.Castilho@rfb.gov.br 790ecec1-bdbc-4b8a-87bd-74e868d2c722 2020-09-14 14:00:00   
leandro.filoni@rfb.gov.br f8074bde-92b5-47f7-a6e9-4acaedb79e05 2020-09-14 14:00:00   
                          c7398489-de13-470f-8f77-a1f9ecd8d9d9 2020-09-14 14:00:00   
                          8d822132-3478-4bbf-a394-b976881db90a 2020-09-14 14:00:00   

action                                                                     termino  
participant_id            session_id                                                
leandro.filoni@rfb.gov.br a82ad53c-0130-49a6-abd6-7bf714973e70 2020-09-14 14:00:00  
Fabio.Castilho@rfb.gov.br 790ecec1-bdbc-4b8a-87bd-74e868d2c722 2020-09-14 14:00:00  
leandro.filoni@rfb.gov.br f8074bde-92b5-47f7-a6e9-4acaedb79e05 2020-09-14 14:00:00  
                          c7398489-de13-470f-8f77-a1f9ecd8d9d9 2020-09-14 14:00:00  
                          8d822132-3478-4bbf-a394-b976881db90a 2020-09-14 14:00:00

,action,Joined,Left,inicio,termino
participant_id,session_id,,,,
hadalton.martinelli@rfb.gov.br,48c47ced-2146-4739-a55c-09fc30355e1e,2020-09-15 07:37:31,2020-09-15 07:50:15,2020-09-14 18:00:00,2020-09-14 18:00:00
Marco.A.Holcman@rfb.gov.br,1bd04d13-4f84-4d26-b1df-2cc2f86bd1d1,2020-09-15 07:45:02,2020-09-15 07:51:28,2020-09-14 18:00:00,2020-09-14 18:00:00
hadalton.martinelli@rfb.gov.br,cc30b0e4-4cca-44ec-87d0-c0460a3b6ddf,2020-09-15 07:49:52,2020-09-15 09:09:00,2020-09-14 18:00:00,2020-09-14 18:00:00
thiago.pizzolato@rfb.gov.br,28de2425-1ca6-4cc1-8f22-bd4c465310ee,2020-09-15 09:08:45,2020-09-15 09:21:56,2020-09-14 18:00:00,2020-09-14 18:00:00
carolina.pinheiro@rfb.gov.br,86033e7c-5bbf-47ed-bdae-79f44f4463ce,2020-09-15 09:10:16,2020-09-14 18:00:00,2020-09-14 18:00:00,2020-09-14 18:00:00


(5823, 4)

In [ ]:
df_final = (
    df_completo
    .reset_index()
    .groupby('participant_id')
    .agg({'inicio':'min', 'termino':'max'})
    .reset_index()
)
d_pd(df_final, n=5)

,participant_id,inicio,termino
0,ANA.BEJAR@rfb.gov.br,2020-09-14 14:30:11,2020-09-14 17:53:09
1,Abrahao.Lima@rfb.gov.br,2020-09-14 14:00:00,2020-09-14 18:00:00
2,Adalto.Goncalves@rfb.gov.br,2020-09-14 14:00:54,2020-09-14 17:59:24
3,Adauri.Souza@rfb.gov.br,2020-09-14 15:19:11,2020-09-14 18:00:00
4,Adelino.Maeda@rfb.gov.br,2020-09-14 15:53:27,2020-09-14 17:58:09
2818,wilson.m.rodrigues-junior@rfb.gov.br,2020-09-14 14:10:28,2020-09-14 16:36:34
2819,yoshiaki.fujinohara@rfb.gov.br,2020-09-14 14:00:00,2020-09-14 14:00:00
2820,yuri-gagharin.braga@rfb.gov.br,2020-09-14 14:12:30,2020-09-14 17:59:35
2821,yuri.godoi@rfb.gov.br,2020-09-14 14:00:00,2020-09-14 16:57:06
2822,zilda.h.pires@rfb.gov.br,2020-09-14 14:00:03,2020-09-14 18:00:00


(2823, 3)

In [ ]:
nome_xlsx = (f"{dct_horarios['Joined']:%Y-%m-%d %H-%M-%S} - "
             f"{dct_horarios['Left']:%Y-%m-%d %H-%M-%S}.xlsx")
nome_xlsx

'2020-09-14 14-00-00 - 2020-09-14 18-00-00.xlsx'

In [ ]:
df_final.to_excel(nome_xlsx, index=False)